
# Descomprimir MetStation y reunir archivos `.25O` por estación


In [1]:

# === CONFIGURACIÓN DE RUTAS (SWIMAC) ===
from pathlib import Path

# Carpeta con los .zip de SWIMAC
ZIP_ROOT = Path(r"C:\Users\varga\Box\Data\RinexZipFiles\SWIMAC\Last")

# Carpeta base donde se crearán subcarpetas por estación (nombre del zip sin extensión)
DEST_ROOT = Path(r"C:\Users\varga\Box\Data\nc Files\SWIMAC")

# Subcadena para identificar la carpeta RINEX (como en tu flujo original)
RINEX_SUBSTR = "RINEX_3_03"

# Patrón de zip a considerar (ajustalo si tus zips llevan otro prefijo)
ZIP_PATTERN = "SWIMAC*.zip"

# Carpeta donde se extraerán los zips (se crea junto a ZIP_ROOT)
UNZIP_ROOT = ZIP_ROOT / "unzipped"

# Política de sobre-escritura de .25O
OVERWRITE_25O = False

ZIP_ROOT, DEST_ROOT, UNZIP_ROOT


(WindowsPath('C:/Users/varga/Box/Data/RinexZipFiles/SWIMAC/Last'),
 WindowsPath('C:/Users/varga/Box/Data/nc Files/SWIMAC'),
 WindowsPath('C:/Users/varga/Box/Data/RinexZipFiles/SWIMAC/Last/unzipped'))

In [ ]:
# === LÓGICA: extraer zips y copiar TODOS los *.25O a una sola carpeta (SWIMAC) ===
import zipfile, shutil
from pathlib import Path

def safe_extract(zf: zipfile.ZipFile, dest: Path):
    for member in zf.infolist():
        member_path = dest / member.filename
        if not str(member_path.resolve()).startswith(str(dest.resolve())):
            raise RuntimeError(f"Entrada peligrosa en ZIP: {member.filename}")
        zf.extract(member, dest)

def extract_all_zips(root: Path, unzip_root: Path, pattern: str):
    unzip_root.mkdir(parents=True, exist_ok=True)
    zips = sorted(root.glob(pattern), key=lambda p: p.name.lower())
    extracted = []
    for z in zips:
        target_dir = unzip_root / z.stem
        if target_dir.exists():
            extracted.append(target_dir)
            continue
        try:
            with zipfile.ZipFile(z) as zf:
                print(f"[EXTRACT] {z.name} -> {target_dir.name}")
                target_dir.mkdir(parents=True, exist_ok=True)
                safe_extract(zf, target_dir)
                extracted.append(target_dir)
        except Exception as e:
            print(f"[ERROR] {z.name}: {e}")
    return extracted

def find_rinex_dirs(base: Path, substr: str):
    sub = substr.lower()
    return [p for p in base.rglob("*") if p.is_dir() and sub in p.name.lower()]

def copy_25O_flat(unzip_root: Path, dest_root: Path, rinex_substr: str, overwrite=False):
    dest_root.mkdir(parents=True, exist_ok=True)
    total = 0
    for station_dir in sorted(p for p in unzip_root.iterdir() if p.is_dir()):
        rinex_dirs = find_rinex_dirs(station_dir, rinex_substr)
        if rinex_dirs:
            for rdir in rinex_dirs:
                for f in rdir.glob("*.25O"):
                    df = dest_root / f.name
                    if df.exists() and not overwrite:
                        continue
                    shutil.copy2(f, df)
                    total += 1
        else:
            for f in station_dir.rglob("*.25O"):
                df = dest_root / f.name
                if df.exists() and not overwrite:
                    continue
                shutil.copy2(f, df)
                total += 1
    return total

# === Ejecutar flujo ===
DEST_ROOT.mkdir(parents=True, exist_ok=True)
extracted_dirs = extract_all_zips(ZIP_ROOT, UNZIP_ROOT, ZIP_PATTERN)
for p in sorted(UNZIP_ROOT.iterdir() if UNZIP_ROOT.exists() else []):
    if p.is_dir() and p not in extracted_dirs:
        extracted_dirs.append(p)

total_copiados = copy_25O_flat(UNZIP_ROOT, DEST_ROOT, RINEX_SUBSTR, overwrite=OVERWRITE_25O)
print("\n===== RESUMEN (Notebook) =====")
print(f"Zips procesados/existentes: {len(extracted_dirs)}")
print(f"Archivos *.25O copiados en total: {total_copiados}")
print(f"Destino único: {DEST_ROOT}")


[EXTRACT] SWIMAC_20250413155647.zip -> SWIMAC_20250413155647
[EXTRACT] SWIMAC_20250414155648.zip -> SWIMAC_20250414155648
[EXTRACT] SWIMAC_20250415155649.zip -> SWIMAC_20250415155649
[EXTRACT] SWIMAC_20250415213723.zip -> SWIMAC_20250415213723
[EXTRACT] SWIMAC_20250415214542.zip -> SWIMAC_20250415214542
[EXTRACT] SWIMAC_20250416214542.zip -> SWIMAC_20250416214542
[EXTRACT] SWIMAC_20250417214543.zip -> SWIMAC_20250417214543
[EXTRACT] SWIMAC_20250418214544.zip -> SWIMAC_20250418214544
[EXTRACT] SWIMAC_20250419214545.zip -> SWIMAC_20250419214545
[EXTRACT] SWIMAC_20250420214546.zip -> SWIMAC_20250420214546
[EXTRACT] SWIMAC_20250421214548.zip -> SWIMAC_20250421214548
[EXTRACT] SWIMAC_20250422214549.zip -> SWIMAC_20250422214549
[EXTRACT] SWIMAC_20250423214550.zip -> SWIMAC_20250423214550
[EXTRACT] SWIMAC_20250424214551.zip -> SWIMAC_20250424214551
[EXTRACT] SWIMAC_20250425214552.zip -> SWIMAC_20250425214552
[EXTRACT] SWIMAC_20250426214553.zip -> SWIMAC_20250426214553
[EXTRACT] SWIMAC_2025042

## Libraries

In [3]:
%load_ext autoreload
%autoreload 2
import gnssvod as gv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pdb
import importlib
import zipfile
import os
import xarray as xr
import glob
import shutil
from gnssvod.hemistats.hemistats import hemibuild
#import georinex as gr
#import qgrid as interactive table 
from matplotlib.collections import PatchCollection
import matplotlib.dates as mdates
import shutil
from matplotlib import rcParams
from datetime import datetime 
import shutil
from pathlib import Path


## Presets

### Time intervals

In [6]:
# Define the start day
startday = pd.to_datetime('25-04-2025', format='%d-%m-%Y')
# Generate a range of datetime values
timeintervals=pd.interval_range(start=startday, periods=10, freq='D', closed='left')
timeintervals

IntervalIndex([[2025-04-25 00:00:00, 2025-04-26 00:00:00),
               [2025-04-26 00:00:00, 2025-04-27 00:00:00),
               [2025-04-27 00:00:00, 2025-04-28 00:00:00),
               [2025-04-28 00:00:00, 2025-04-29 00:00:00),
               [2025-04-29 00:00:00, 2025-04-30 00:00:00),
               [2025-04-30 00:00:00, 2025-05-01 00:00:00),
               [2025-05-01 00:00:00, 2025-05-02 00:00:00),
               [2025-05-02 00:00:00, 2025-05-03 00:00:00),
               [2025-05-03 00:00:00, 2025-05-04 00:00:00),
               [2025-05-04 00:00:00, 2025-05-05 00:00:00)],
              dtype='interval[datetime64[ns], left]')

## Process Rinex Files

In [4]:
from pathlib import Path

# Carpeta única donde se copiaron todos los .25O de SWIMAC
root = Path(r"C:\Users\varga\Box\Data\nc Files\SWIMAC")

# Buscar todos los archivos con extensión .25O
files = list(root.glob("*.25O"))

print("Found", len(files), "files:")
for f in files:
    print(f.name)


Found 114 files:
SWIMAC_raw_20250413155647.25O
SWIMAC_raw_20250414155648.25O
SWIMAC_raw_20250415155649.25O
SWIMAC_raw_20250415213723.25O
SWIMAC_raw_20250415214542.25O
SWIMAC_raw_20250416214542.25O
SWIMAC_raw_20250417214543.25O
SWIMAC_raw_20250418214544.25O
SWIMAC_raw_20250419214545.25O
SWIMAC_raw_20250420214546.25O
SWIMAC_raw_20250421214548.25O
SWIMAC_raw_20250422214549.25O
SWIMAC_raw_20250423214550.25O
SWIMAC_raw_20250424214551.25O
SWIMAC_raw_20250425214552.25O
SWIMAC_raw_20250426214553.25O
SWIMAC_raw_20250427214554.25O
SWIMAC_raw_20250428214555.25O
SWIMAC_raw_20250429214556.25O
SWIMAC_raw_20250430214557.25O
SWIMAC_raw_20250501214558.25O
SWIMAC_raw_20250502214559.25O
SWIMAC_raw_20250503214600.25O
SWIMAC_raw_20250504214601.25O
SWIMAC_raw_20250505214602.25O
SWIMAC_raw_20250524160855.25O
SWIMAC_raw_20250524161117.25O
SWIMAC_raw_20250524162917.25O
SWIMAC_raw_20250525153357.25O
SWIMAC_raw_20250526103916.25O
SWIMAC_raw_20250527103917.25O
SWIMAC_raw_20250528103918.25O
SWIMAC_raw_202505291039

In [1]:
from pathlib import Path
import glob
import os
import xarray as xr
import gnssvod as gv

# ==== Config ====
FORCE_REDO = False  # ponelo en True si querés reprocesar aunque ya exista salida

# Carpeta donde dejaste TODOS los .25O de SWIMAC
DEST = Path(r"C:\Users\varga\Box\Data\nc Files\SWIMAC")

# 1) Lista de archivos de entrada (RINEX .25O)
rinex_files = sorted(glob.glob(str(DEST / '*.25O')))
print(f'Found {len(rinex_files)} .25O files (showing up to 5):', rinex_files[:5])
if len(rinex_files) == 0:
    raise FileNotFoundError("No .25O files found in DEST folder. Check DEST and file existence.")

# 2) Carpeta de salida para NetCDF (dentro de SWIMAC)
outdir = DEST / "processed"
outdir.mkdir(parents=True, exist_ok=True)

# 3) Carpeta persistente para órbitas/relojes (evita fallos por SP3/CLK)
aux_dir = DEST / "aux_orbits"
aux_dir.mkdir(parents=True, exist_ok=True)

# 4) Variables a conservar (SNR)
keepvars = ['S?', 'S??']

# ---- utilidades para decidir si ya está procesado ----
def has_output_for(stem: str) -> bool:
    """
    Devuelve True si ya existe al menos un archivo de salida para este stem
    en 'processed/' con extensión .nc o un directorio .zarr.
    """
    nc_hits = list(outdir.glob(f"{stem}*.nc"))
    zarr_hits = list(outdir.glob(f"{stem}*.zarr"))
    return (len(nc_hits) + len(zarr_hits)) > 0

def needs_processing(rinex_path: str) -> bool:
    """
    Si FORCE_REDO es False: salta si ya hay salida.
    Si FORCE_REDO es True: siempre procesa.
    (Opcional: podrías comparar mtime para re-procesar si el .25O es más nuevo).
    """
    if FORCE_REDO:
        return True
    stem = Path(rinex_path).stem
    return not has_output_for(stem)

ok, bad, skipped = [], [], []

# 5) Procesar uno por uno con control de errores y skip de ya-procesados
for rinex in rinex_files:
    stem = Path(rinex).stem

    if not needs_processing(rinex):
        print(f"[SKIP] {stem} (ya existe salida en '{outdir.name}')")
        skipped.append(rinex)
        continue

    try:
        # patrón y salida para este archivo en particular
        pattern = {'SWIMAC': rinex}
        outputdir = {'SWIMAC': str(outdir)}

        result = gv.preprocess(
            pattern,
            interval='15s',
            keepvars=keepvars,
            outputdir=outputdir,
            outputresult=True,
            aux_path=str(aux_dir)  # clave para cachear SP3/CLK
        )

        # Validación simple del resultado
        swimac_res = result.get('SWIMAC', [])
        if isinstance(swimac_res, list):
            if not swimac_res:
                raise ValueError(f"No dataset produced for: {rinex}")
            _ = swimac_res[0]
        elif isinstance(swimac_res, xr.Dataset):
            _ = swimac_res
        else:
            raise TypeError(f"Unexpected return type for 'SWIMAC': {type(swimac_res)}")

        print(f"[OK] {Path(rinex).name}")
        ok.append(rinex)

    except Exception as e:
        print(f"[WARN] Skipping due to error: {Path(rinex).name}")
        print("       ", e)
        bad.append((rinex, str(e)))
        continue

# 6) Resumen final
print("\n===== SUMMARY =====")
print(f"Processed OK: {len(ok)}")
print(f"Skipped    : {len(skipped)} (ya tenían salida)")
print(f"Failed     : {len(bad)}")

if bad:
    print("\nFiles that failed:")
    for fname, err in bad:
        print(f" - {fname}: {err}")
else:
    print("\n¡Todo fue un éxito! 🤙")


Found 114 .25O files (showing up to 5): ['C:\\Users\\varga\\Box\\Data\\nc Files\\SWIMAC\\SWIMAC_raw_20250413155647.25O', 'C:\\Users\\varga\\Box\\Data\\nc Files\\SWIMAC\\SWIMAC_raw_20250414155648.25O', 'C:\\Users\\varga\\Box\\Data\\nc Files\\SWIMAC\\SWIMAC_raw_20250415155649.25O', 'C:\\Users\\varga\\Box\\Data\\nc Files\\SWIMAC\\SWIMAC_raw_20250415213723.25O', 'C:\\Users\\varga\\Box\\Data\\nc Files\\SWIMAC\\SWIMAC_raw_20250415214542.25O']
[SKIP] SWIMAC_raw_20250413155647 (ya existe salida en 'processed')
Could not find any files matching the pattern C:\Users\varga\Box\Data\nc Files\SWIMAC\processed*.nc
C:\Users\varga\Box\Data\nc Files\SWIMAC\SWIMAC_raw_20250414155648.25O exists | Reading...
Processed 393694 out of 3181366 lines (12.4%)
Processed 1202344 out of 3181366 lines (37.8%)
Processed 1983117 out of 3181366 lines (62.3%)
Processed 2511957 out of 3181366 lines (79.0%)
Processed 3043641 out of 3181366 lines (95.7%)
Observation file  C:\Users\varga\Box\Data\nc Files\SWIMAC\SWIMAC_raw

GFZ0MGXRAP_20251050000_01D_05M_ORB.SP3.gz: 1.04MB [00:03, 312kB/s]                             


 | Download completed for ftp://gssc.esa.int/gnss/products/2362//GFZ0MGXRAP_20251050000_01D_05M_ORB.SP3.gz
C:\Users\varga\Box\Data\nc Files\SWIMAC\aux_orbits\GFZ0MGXRAP_20251050000_01D_05M_ORB.SP3 file is read in 10.26 seconds
C:\Users\varga\Box\Data\nc Files\SWIMAC\aux_orbits\GFZ0MGXRAP_20251040000_01D_30S_CLK.CLK exists | Reading...
C:\Users\varga\Box\Data\nc Files\SWIMAC\aux_orbits\GFZ0MGXRAP_20251040000_01D_30S_CLK.CLK file is read in 13.91 seconds
This file does not exist: C:\Users\varga\Box\Data\nc Files\SWIMAC\aux_orbits\GFZ0MGXRAP_20251050000_01D_30S_CLK.CLK
Downloading: GFZ0MGXRAP_20251050000_01D_30S_CLK.CLK.gz

GFZ0MGXRAP_20251050000_01D_30S_CLK.CLK.gz: 4.80MB [00:03, 1.36MB/s]                            


 | Download completed for GFZ0MGXRAP_20251050000_01D_30S_CLK.CLK.gz
C:\Users\varga\Box\Data\nc Files\SWIMAC\aux_orbits\GFZ0MGXRAP_20251050000_01D_30S_CLK.CLK file is read in 1.86 seconds
SP3 interpolation is done in 3.16 seconds
Saved 210381 individual observations in C:\Users\varga\Box\Data\nc Files\SWIMAC\processed\SWIMAC_raw_20250414155648.nc
[OK] SWIMAC_raw_20250414155648.25O
Could not find any files matching the pattern C:\Users\varga\Box\Data\nc Files\SWIMAC\processed*.nc
C:\Users\varga\Box\Data\nc Files\SWIMAC\SWIMAC_raw_20250415155649.25O exists | Reading...
Processed 545127 out of 782076 lines (69.7%)
Observation file  C:\Users\varga\Box\Data\nc Files\SWIMAC\SWIMAC_raw_20250415155649.25O  is read in 39.60 seconds.
Processing 761642 individual observations
Calculating Azimuth and Elevation
C:\Users\varga\Box\Data\nc Files\SWIMAC\aux_orbits\GFZ0MGXRAP_20251050000_01D_05M_ORB.SP3 exists | Reading...
C:\Users\varga\Box\Data\nc Files\SWIMAC\aux_orbits\GFZ0MGXRAP_20251050000_01D_05M

...

 | Download completed for GFZ0MGXRAP_20252340000_01D_30S_CLK.CLK.gz
C:\Users\varga\Box\Data\nc Files\SWIMAC\aux_orbits\GFZ0MGXRAP_20252340000_01D_30S_CLK.CLK file is read in 1.97 seconds
SP3 interpolation is done in 2.46 seconds
Saved 103146 individual observations in C:\Users\varga\Box\Data\nc Files\SWIMAC\processed\SWIMAC_raw_20250821145739.nc
[OK] SWIMAC_raw_20250821145739.25O
Could not find any files matching the pattern C:\Users\varga\Box\Data\nc Files\SWIMAC\processed*.nc
C:\Users\varga\Box\Data\nc Files\SWIMAC\SWIMAC_raw_20250822145740.25O exists | Reading...
Observation file  C:\Users\varga\Box\Data\nc Files\SWIMAC\SWIMAC_raw_20250822145740.25O  is read in 4.72 seconds.
Processing 99093 individual observations
Calculating Azimuth and Elevation
C:\Users\varga\Box\Data\nc Files\SWIMAC\aux_orbits\GFZ0MGXRAP_20252340000_01D_05M_ORB.SP3 exists | Reading...
C:\Users\varga\Box\Data\nc Files\SWIMAC\aux_orbits\GFZ0MGXRAP_20252340000_01D_05M_ORB.SP3 file is read in 0.72 seconds
This file

GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3.gz: 0.00B [00:02, ?B/s]


<urlopen error <urlopen error ftp error: error_perm('550 CWD command failed: directory not found.')>>
 | Requested file ftp://gssc.esa.int/gnss/products/2380//GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3.gz cannot be not found!
[WARN] Skipping due to error: SWIMAC_raw_20250822145740.25O
        [Errno 2] No such file or directory: 'C:\\Users\\varga\\Box\\Data\\nc Files\\SWIMAC\\aux_orbits\\GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3'
Could not find any files matching the pattern C:\Users\varga\Box\Data\nc Files\SWIMAC\processed*.nc
C:\Users\varga\Box\Data\nc Files\SWIMAC\SWIMAC_raw_20250823145741.25O exists | Reading...
Observation file  C:\Users\varga\Box\Data\nc Files\SWIMAC\SWIMAC_raw_20250823145741.25O  is read in 0.90 seconds.
Processing 25039 individual observations
Calculating Azimuth and Elevation
This file does not exist: C:\Users\varga\Box\Data\nc Files\SWIMAC\aux_orbits\GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3
Downloading: ftp://gssc.esa.int/gnss/products/2380//GFZ0MGXRAP_20252350000_01D

GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3.gz: 0.00B [00:02, ?B/s]

<urlopen error <urlopen error ftp error: error_perm('550 CWD command failed: directory not found.')>>
 | Requested file ftp://gssc.esa.int/gnss/products/2380//GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3.gz cannot be not found!
[WARN] Skipping due to error: SWIMAC_raw_20250823145741.25O
        [Errno 2] No such file or directory: 'C:\\Users\\varga\\Box\\Data\\nc Files\\SWIMAC\\aux_orbits\\GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3'

===== SUMMARY =====
Processed OK: 111
Skipped    : 1 (ya tenían salida)
Failed     : 2

Files that failed:
 - C:\Users\varga\Box\Data\nc Files\SWIMAC\SWIMAC_raw_20250822145740.25O: [Errno 2] No such file or directory: 'C:\\Users\\varga\\Box\\Data\\nc Files\\SWIMAC\\aux_orbits\\GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3'
 - C:\Users\varga\Box\Data\nc Files\SWIMAC\SWIMAC_raw_20250823145741.25O: [Errno 2] No such file or directory: 'C:\\Users\\varga\\Box\\Data\\nc Files\\SWIMAC\\aux_orbits\\GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3'
